In [35]:
import numpy as np
import csv
from collections import defaultdict
from sklearn import linear_model
import pandas as pd

In [36]:
def csv_to_list(file_name):
    d = []
    with open(file_name, newline='') as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            d.append({'user_id': line[0], 'recipe_id': line[1], 'date': line[2], 'rating': line[3], 'u': line[4], 'i': line[5]})
    return d[1:]

def MSE(y, ypred):
    return sum(x**2 for x in (y - ypred)) / len(y)

In [37]:
data_train = csv_to_list('interactions_train.csv')
data_valid = csv_to_list('interactions_validation.csv')
data_test = csv_to_list('interactions_test.csv')
test_Y = [float(d['rating']) for d in data_test]

In [38]:
sum([float(d['rating']) for d in data_train]) / len(data_train)

4.574089892559891

In [39]:
global_average_rating = round(sum([float(d['rating']) for d in data_train]) / len(data_train))
global_average_rating

5

In [40]:
# Logistic Regression with Number of Steps Only

In [41]:
#PP_recipes contains information for every recipe
pp_recipes  = pd.read_csv('PP_recipes.csv')

In [42]:
pp_steps = pp_recipes['steps_tokens']
temp = pp_recipes['id']
ids = [str(num) for num in temp]
len_steps = [len(d) for d in pp_steps]

In [43]:
#Map every recipe ID to its number of steps
id_to_steps = dict(zip(ids, len_steps))

In [44]:
X_train = [[1 ,id_to_steps[d['recipe_id']]] for d in data_train]
Y_train = [float(d['rating']) for d in data_train]

In [45]:
model =linear_model.LinearRegression ( fit_intercept = False)
model.fit(X_train,Y_train)
theta = model.coef_
theta

array([4.55585841e+00, 2.64976818e-05])

In [46]:
#Use Step Number Regression Model on Test Set
predict_Y = [0] * len(data_test)

for i in range(len(data_test)):
    recipe = data_test[i]['recipe_id']
    if recipe in id_to_steps:
        predict_Y[i] = theta[0]+ theta[1]*id_to_steps[recipe]
    else:
        predict_Y[i] = 5


In [47]:
MSE_list = {}
mse = MSE(test_Y, np.array(predict_Y))
print(mse)
MSE_list['steps'] = mse

1.922033907295291


In [48]:
# Logistic Regression with Number of Ingredients Only
pp_Ing = pp_recipes['ingredient_tokens']
temp = pp_recipes['id']
ids = [str(num) for num in temp]
len_Ing = [len(d) for d in pp_Ing]

In [49]:
#Map every recipe ID to its number of steps
id_to_ingredients = dict(zip(ids, len_Ing))

In [50]:
X_train = [[1 ,id_to_ingredients[d['recipe_id']]] for d in data_train]
Y_train = [float(d['rating']) for d in data_train]

In [51]:
model =linear_model.LinearRegression ( fit_intercept = False)
model.fit(X_train,Y_train)
theta = model.coef_
theta

array([4.56593736e+00, 5.71766110e-05])

In [52]:
#Use Ingredient Number Regression Model on Test Set
predict_Y = [0] * len(data_test)

for i in range(len(data_test)):
    recipe = data_test[i]['recipe_id']
    if recipe in id_to_ingredients:
        predict_Y[i] = theta[0]+ theta[1]*id_to_ingredients[recipe]
    else:
        predict_Y[i] = 5


In [53]:
mse = MSE(test_Y, np.array(predict_Y))
print(mse)
MSE_list['ing'] = mse

1.9217262790108383


In [54]:
#Logistic Regression with Both Steps and Ingredients

In [55]:
X_train = [[1 ,id_to_ingredients[d['recipe_id']], id_to_steps[d['recipe_id']]] for d in data_train]
Y_train = [float(d['rating']) for d in data_train]

In [56]:
model =linear_model.LinearRegression ( fit_intercept = False)
model.fit(X_train,Y_train)
theta = model.coef_
theta

array([4.55530180e+00, 5.82835989e-06, 2.60988183e-05])

In [57]:
#Use Ingredient Number Regression Model on Test Set
predict_Y = [0] * len(data_test)

for i in range(len(data_test)):
    recipe = data_test[i]['recipe_id']
    if recipe in id_to_ingredients:
        predict_Y[i] = theta[0]+ theta[1]*id_to_ingredients[recipe] + theta[2] * id_to_steps[recipe]
    else:
        predict_Y[i] = 5

In [58]:
MSE(test_Y, np.array(predict_Y))
MSE_list['steps and ing'] = MSE(test_Y, np.array(predict_Y))

In [59]:
print(MSE_list)

{'steps': 1.922033907295291, 'ing': 1.9217262790108383, 'steps and ing': 1.9220255109190323}
